# About this kernel

I've seen a lot of people pooling the output of BERT, then add some Dense layers. I also saw various learning rates for fine-tuning. In this kernel, I wanted to try some ideas that were used in the original paper that did not appear in many public kernel. Here are some examples:
* *No pooling, directly use the CLS embedding*. The original paper uses the output embedding for the `[CLS]` token when it is finetuning for classification tasks, such as sentiment analysis. Since the `[CLS]` token is the first token in our sequence, we simply take the first slice of the 2nd dimension from our tensor of shape `(batch_size, max_len, hidden_dim)`, which result in a tensor of shape `(batch_size, hidden_dim)`.
* *No Dense layer*. Simply add a sigmoid output directly to the last layer of BERT, rather than experimenting with different intermediate layers.
* *Fixed learning rate, batch size, epochs, optimizer*. As specified by the paper, the optimizer used is Adam, with a learning rate between 2e-5 and 5e-5. Furthermore, they train the model for 3 epochs with a batch size of 32. I wanted to see how well it would perform with those default values.

I also wanted to share this kernel as a **concise, reusable, and functional example of how to build a workflow around the TF2 version of BERT**. Indeed, it takes less than **50 lines of code to write a string-to-tokens preprocessing function and model builder**.

## References

* Source for `bert_encode` function: https://www.kaggle.com/user123454321/bert-starter-inference
* All pre-trained BERT models from Tensorflow Hub: https://tfhub.dev/s?q=bert

In [1]:
# We will use the official tokenization script created by the Google team
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [2]:
!pip install tensorflow

     |████████████████████████████████| 516.2 MB 14 kB/s s eta 0:00:01    |█████▎                          | 85.8 MB 71.5 MB/s eta 0:00:07     |███████████████████████▏        | 374.6 MB 74.1 MB/s eta 0:00:02     |██████████████████████████▋     | 430.0 MB 67.3 MB/s eta 0:00:02
     |████████████████████████████████| 3.0 MB 67.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 61.7 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 68.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 104 kB 73.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 183 kB/s  eta 0:00:01
     |████████████████████████████████| 91 kB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 2.5 MB/s  eta 0:00:01
     |████████████████████████████████| 779 kB 66.7 MB/s eta 0:00:01
     |██████████████████████████████

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.1 MB 9.5 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install tensorflow_hub

     |████████████████████████████████| 101 kB 8.5 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

# Helper Functions

In [6]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [7]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Load and Preprocess

- Load BERT from the Tensorflow Hub
- Load CSV files containing training data
- Load tokenizer from the bert layer
- Encode the text into tokens, masks, and segment flags

In [8]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 25.8 s, sys: 6.2 s, total: 32 s
Wall time: 45.4 s


In [9]:
train = pd.read_csv("train_clean.csv")
test = pd.read_csv("test_clean.csv")
submission = pd.read_csv("sample_submission.csv")

In [10]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [11]:
#tokenizing the text
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

# Model: Build, Train, Predict, Submit

In [12]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
#training the model
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=5,
    callbacks=[checkpoint],
    batch_size=16
)

Epoch 1/5
381/381 [==============================] - 311s 817ms/step - loss: 0.4189 - accuracy: 0.8189 - val_loss: 0.3796 - val_accuracy: 0.8464
Epoch 2/5
381/381 [==============================] - 213s 560ms/step - loss: 0.2737 - accuracy: 0.8895 - val_loss: 0.4466 - val_accuracy: 0.8345
Epoch 3/5
174/381 [============>.................] - ETA: 1:46 - loss: 0.1357 - accuracy: 0.9537

In [23]:
#with 3 epoch
metrics=pd.DataFrame(model.history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,0.434477,0.811658,0.381478,0.843729
1,0.289927,0.891790,0.380350,0.847012
2,0.208712,0.925123,0.429506,0.835194


In [15]:
#with cleaned data
#with 5 epoch
metrics=pd.DataFrame(model.history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,0.418903,0.818883,0.379638,0.846356
1,0.273738,0.889491,0.446558,0.834537
2,0.132578,0.953366,0.608245,0.820092
3,0.077123,0.973563,0.589503,0.837164
4,0.052858,0.977340,0.608156,0.839133


In [29]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [30]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)